# Santa 2025 - Accelerated Solution
This notebook uses **Numba** JIT compilation to accelerate the geometric calculations and Simulated Annealing process.
It is designed to run significantly faster than the pure Python baseline, allowing for millions of iterations.


In [ ]:
# Check Numba Acceleration Status
import numba
print(f"Numba Version: {numba.__version__}")

# Check if CUDA is available (for information purposes, though we use CPU JIT)
try:
    from numba import cuda
    if cuda.is_available():
        print("CUDA GPU is available!")
        cuda.detect()
    else:
        print("CUDA GPU is NOT available. Using CPU acceleration.")
except ImportError:
    print("Numba CUDA support not installed. Using CPU acceleration.")

# Print Threading Layer Info
try:
    import numba.threading_layer
    print("Threading Layer:", numba.threading_layer.threading_layer())
except:
    pass


In [ ]:
## 2. Setup

import math
import random
import time
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from tqdm.notebook import tqdm

# Set random seed
SEED = 2025
random.seed(SEED)
np.random.seed(SEED)

# High precision for coordinates
getcontext().prec = 50

print("Libraries imported and seed set.")

In [ ]:
# Configuration for Advanced Features
CONFIG = {
    # Hybrid Search Approach
    'hybrid_search': {
        'enabled': True,
        'switch_n': 25,  # Use forward building for N <= 25
    },
    
    # Dynamic Beam Width
    'dynamic_beam_width': {
        'enabled': True,
        'base_width': 12,
        'critical_n_multiplier': 2.0, # Wider for critical sizes
    },
    
    # Beam Search Diversity
    'beam_diversity': {
        'enabled': True,
        'max_children_per_parent': 3, # Prevent one parent from dominating the next generation
    },
    
    # Multi-Start Optimization
    'multi_start': {
        'enabled': True,
        'n_starts': 3,
    },
    
    # Optimization Function Enhancements
    'optimization': {
        'adaptive_temperature': True,
        'smart_move_selection': True, # Bias moves toward boundary
        'batch_moves': False, # Try moving multiple trees (complex)
        'boundary_bias_strength': 0.7, # 70% chance to pick boundary tree
        'reheating': True, # Restart SA if stuck
        'compaction_pass': True, # Pure translation pass to close gaps
    },
    
    # Post-Processing & Polishing
    'post_processing': {
        'polishing': True, # Run a low-temp refinement phase at the end
        'polishing_iterations': 2000,
    },
    
    # Geometric & Heuristic Improvements
    'heuristics': {
        'corner_filling': True,
        'symmetry_breaking': True,
        'boundary_alignment': True,
    },
    
    # Validation
    'validation': {
        'strict_boundary': True,
        'collision_check': True,
    },
    
    # Debugging
    'debug': {
        'verbose_logging': True,
        'plot_convergence': True,
    }
}

print("Configuration loaded:", CONFIG)

In [ ]:
## 3. Load Data

# Load the sample submission to understand the required output format
try:
    sample_sub = pd.read_csv('test.csv')
    print("Sample Submission Shape:", sample_sub.shape)
    print(sample_sub.head())
except FileNotFoundError:
    print("Sample submission not found, proceeding without it.")

## 4. Baseline Strategy

We define the `GreedyPacker` class here. It encapsulates the logic for placing a single tree into an existing configuration.

In [ ]:
class GreedyPacker:
    def __init__(self, n_trials=100, step_size=0.2, fine_step=0.02):
        self.n_trials = n_trials
        self.step_size = step_size
        self.fine_step = fine_step

    def _generate_weighted_angle(self):
        """
        Generates a random angle with a distribution weighted by abs(sin(2*angle)).
        This helps place more trees in corners (diagonals).
        """
        if not CONFIG['heuristics']['corner_filling']:
            return random.uniform(0, 2 * math.pi)
            
        while True:
            angle = random.uniform(0, 2 * math.pi)
            if random.uniform(0, 1) < abs(math.sin(2 * angle)):
                return angle

    def place_next_tree(self, existing_trees, tree_class):
        """Finds the best position for the next tree given existing trees."""
        if not existing_trees:
            return tree_class(0, 0, 0)

        existing_polys = [t.polygon for t in existing_trees]
        tree_index = STRtree(existing_polys)
        
        # Calculate current bounds and center
        minx, miny, maxx, maxy = unary_union(existing_polys).bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2
        
        best_tree = None
        min_metric = float('inf')

        for _ in range(self.n_trials):
            # Random angle for the tree itself
            angle = random.uniform(0, 360)
            
            # Weighted approach angle (bias towards diagonals)
            approach_angle = self._generate_weighted_angle()
            vx, vy = math.cos(approach_angle), math.sin(approach_angle)
            
            # Start far away
            radius = max(maxx - minx, maxy - miny) + 10.0
            candidate = tree_class(0, 0, angle)
            
            # Move in
            current_r = radius
            collision = False
            
            # Coarse search
            while current_r > 0:
                px, py = center_x + current_r * vx, center_y + current_r * vy
                candidate.update_position(px, py, angle)
                
                query_indices = tree_index.query(candidate.polygon)
                if any(candidate.polygon.intersects(existing_polys[i]) for i in query_indices):
                    collision = True
                    break
                current_r -= self.step_size
            
            # Fine tune
            if collision:
                current_r += self.step_size
                while True:
                    current_r -= self.fine_step
                    px, py = center_x + current_r * vx, center_y + current_r * vy
                    candidate.update_position(px, py, angle)
                    
                    query_indices = tree_index.query(candidate.polygon)
                    if any(candidate.polygon.intersects(existing_polys[i]) for i in query_indices):
                        # Collision found, step back once and stop
                        current_r += self.fine_step
                        px, py = center_x + current_r * vx, center_y + current_r * vy
                        candidate.update_position(px, py, angle)
                        break
            else:
                candidate.update_position(center_x, center_y, angle)

            # Metric: Minimize the side length of the new bounding box
            t_minx, t_miny, t_maxx, t_maxy = candidate.polygon.bounds
            new_minx = min(minx, t_minx)
            new_miny = min(miny, t_miny)
            new_maxx = max(maxx, t_maxx)
            new_maxy = max(maxy, t_maxy)
            
            new_side = max(new_maxx - new_minx, new_maxy - new_miny)
            
            # Tie-breaker: distance to center
            dist_sq = (px - center_x)**2 + (py - center_y)**2
            
            metric = new_side + (dist_sq * 1e-6)
            
            if metric < min_metric:
                min_metric = metric
                best_tree = tree_class(px, py, angle)
                
        return best_tree

## 5. Feature Engineering Module

Here we define the geometric features of the problem: the `ChristmasTree` class and helper functions for bounding boxes.

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""
    def __init__(self, center_x=0, center_y=0, angle=0):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self.polygon = self._create_polygon()

    def _create_polygon(self):
        # Tree dimensions
        coords = [
            (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
            (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
            (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
        ]
        poly = Polygon(coords)
        rotated = affinity.rotate(poly, self.angle, origin=(0, 0))
        return affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

    def update_position(self, x, y, angle):
        self.center_x = x
        self.center_y = y
        self.angle = angle
        self.polygon = self._create_polygon()

def get_bounds(trees):
    if not trees: return 0
    minx, miny, maxx, maxy = unary_union([t.polygon for t in trees]).bounds
    return max(maxx - minx, maxy - miny)

In [ ]:
# --- NUMBA ACCELERATED GEOMETRY KERNEL ---
from numba import njit
import numpy as np

# Tree polygon vertices (Same as defined in ChristmasTree class)
TREE_X = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075,
                   -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125], dtype=np.float64)
TREE_Y = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2,
                   -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5], dtype=np.float64)
NV = 15

@njit(cache=True, fastmath=True)
def get_poly(cx, cy, deg):
    rad = deg * np.pi / 180.0
    c, s = np.cos(rad), np.sin(rad)
    px = TREE_X * c - TREE_Y * s + cx
    py = TREE_X * s + TREE_Y * c + cy
    return px, py

@njit(cache=True, fastmath=True)
def get_bbox(px, py):
    return px.min(), py.min(), px.max(), py.max()

@njit(cache=True, fastmath=True)
def pip(px_pt, py_pt, poly_x, poly_y):
    inside = False
    j = NV - 1
    for i in range(NV):
        if ((poly_y[i] > py_pt) != (poly_y[j] > py_pt) and
            px_pt < (poly_x[j] - poly_x[i]) * (py_pt - poly_y[i]) / (poly_y[j] - poly_y[i]) + poly_x[i]):
            inside = not inside
        j = i
    return inside

@njit(cache=True, fastmath=True)
def seg_intersect(ax, ay, bx, by, cx, cy, dx, dy):
    def ccw(p1x, p1y, p2x, p2y, p3x, p3y):
        return (p3y - p1y) * (p2x - p1x) > (p2y - p1y) * (p3x - p1x)
    return ccw(ax, ay, cx, cy, dx, dy) != ccw(bx, by, cx, cy, dx, dy) and \
           ccw(ax, ay, bx, by, cx, cy) != ccw(ax, ay, bx, by, dx, dy)

@njit(cache=True, fastmath=True)
def overlap(px1, py1, bb1, px2, py2, bb2):
    # Fast BBox check
    if bb1[2] < bb2[0] or bb2[2] < bb1[0] or bb1[3] < bb2[1] or bb2[3] < bb1[1]:
        return False
    # Point in Polygon check
    for i in range(NV):
        if pip(px1[i], py1[i], px2, py2): return True
        if pip(px2[i], py2[i], px1, py1): return True
    # Edge Intersection check
    for i in range(NV):
        ni = (i + 1) % NV
        for j in range(NV):
            nj = (j + 1) % NV
            if seg_intersect(px1[i], py1[i], px1[ni], py1[ni],
                           px2[j], py2[j], px2[nj], py2[nj]):
                return True
    return False

@njit(cache=True, fastmath=True)
def check_overlap_single(idx, xs, ys, angs, n):
    px1, py1 = get_poly(xs[idx], ys[idx], angs[idx])
    bb1 = get_bbox(px1, py1)
    for j in range(n):
        if j != idx:
            px2, py2 = get_poly(xs[j], ys[j], angs[j])
            bb2 = get_bbox(px2, py2)
            if overlap(px1, py1, bb1, px2, py2, bb2):
                return True
    return False

@njit(cache=True, fastmath=True)
def check_overlap_pair(i, j, xs, ys, angs, n):
    pxi, pyi = get_poly(xs[i], ys[i], angs[i])
    pxj, pyj = get_poly(xs[j], ys[j], angs[j])
    bbi = get_bbox(pxi, pyi)
    bbj = get_bbox(pxj, pyj)
    if overlap(pxi, pyi, bbi, pxj, pyj, bbj):
        return True
    for k in range(n):
        if k != i and k != j:
            pxk, pyk = get_poly(xs[k], ys[k], angs[k])
            bbk = get_bbox(pxk, pyk)
            if overlap(pxi, pyi, bbi, pxk, pyk, bbk):
                return True
            if overlap(pxj, pyj, bbj, pxk, pyk, bbk):
                return True
    return False

@njit(cache=True, fastmath=True)
def calc_side(xs, ys, angs, n):
    if n == 0: return 0.0
    gx0, gy0, gx1, gy1 = 1e9, 1e9, -1e9, -1e9
    for i in range(n):
        px, py = get_poly(xs[i], ys[i], angs[i])
        x0, y0, x1, y1 = get_bbox(px, py)
        gx0, gy0 = min(gx0, x0), min(gy0, y0)
        gx1, gy1 = max(gx1, x1), max(gy1, y1)
    return max(gx1 - gx0, gy1 - gy0)

@njit(cache=True, fastmath=True)
def get_global_bbox(xs, ys, angs, n):
    gx0, gy0, gx1, gy1 = 1e9, 1e9, -1e9, -1e9
    for i in range(n):
        px, py = get_poly(xs[i], ys[i], angs[i])
        x0, y0, x1, y1 = get_bbox(px, py)
        gx0, gy0 = min(gx0, x0), min(gy0, y0)
        gx1, gy1 = max(gx1, x1), max(gy1, y1)
    return gx0, gy0, gx1, gy1

@njit(cache=True, fastmath=True)
def find_corner_trees(xs, ys, angs, n):
    gx0, gy0, gx1, gy1 = get_global_bbox(xs, ys, angs, n)
    eps = 0.01
    # We can't return dynamic arrays easily in njit without some work, 
    # so we return a fixed size array and a count, or just use a list
    # For simplicity, let's return indices in a fixed buffer
    corner_indices = np.zeros(n, dtype=np.int32)
    count = 0
    for i in range(n):
        px, py = get_poly(xs[i], ys[i], angs[i])
        x0, y0, x1, y1 = get_bbox(px, py)
        if abs(x0 - gx0) < eps or abs(x1 - gx1) < eps or \
           abs(y0 - gy0) < eps or abs(y1 - gy1) < eps:
            corner_indices[count] = i
            count += 1
    return corner_indices, count

@njit(cache=True, fastmath=True)
def sa_numba(xs, ys, angs, n, iterations, T0, Tmin, move_scale, rot_scale, seed):
    np.random.seed(seed)

    bxs, bys, bangs = xs.copy(), ys.copy(), angs.copy()
    cxs, cys, cangs = xs.copy(), ys.copy(), angs.copy()

    bs = calc_side(bxs, bys, bangs, n)
    cs = bs
    T = T0
    alpha = (Tmin / T0) ** (1.0 / iterations) if iterations > 0 else 0.99
    no_imp = 0

    for it in range(iterations):
        move_type = np.random.randint(0, 8)  # 8 move types
        sc = T / T0

        if move_type < 4:
            # Single tree moves
            i = np.random.randint(0, n)
            ox, oy, oa = cxs[i], cys[i], cangs[i]

            cx = np.mean(cxs[:n])
            cy = np.mean(cys[:n])

            if move_type == 0: # Random translation
                cxs[i] += (np.random.random() - 0.5) * 2 * move_scale * sc
                cys[i] += (np.random.random() - 0.5) * 2 * move_scale * sc
            elif move_type == 1: # Move towards center
                dx, dy = cx - cxs[i], cy - cys[i]
                d = np.sqrt(dx*dx + dy*dy)
                if d > 1e-6:
                    step = np.random.random() * move_scale * sc
                    cxs[i] += dx / d * step
                    cys[i] += dy / d * step
            elif move_type == 2: # Rotation
                cangs[i] += (np.random.random() - 0.5) * 2 * rot_scale * sc
                cangs[i] = cangs[i] % 360
            else: # Mixed
                cxs[i] += (np.random.random() - 0.5) * move_scale * sc
                cys[i] += (np.random.random() - 0.5) * move_scale * sc
                cangs[i] += (np.random.random() - 0.5) * rot_scale * sc
                cangs[i] = cangs[i] % 360

            if check_overlap_single(i, cxs, cys, cangs, n):
                cxs[i], cys[i], cangs[i] = ox, oy, oa
                no_imp += 1
                T *= alpha
                if T < Tmin: T = Tmin
                continue

        elif move_type == 4 and n > 1:
            # Swap
            i = np.random.randint(0, n)
            j = np.random.randint(0, n)
            while j == i: j = np.random.randint(0, n)

            oxi, oyi = cxs[i], cys[i]
            oxj, oyj = cxs[j], cys[j]

            cxs[i], cys[i] = oxj, oyj
            cxs[j], cys[j] = oxi, oyi

            if check_overlap_pair(i, j, cxs, cys, cangs, n):
                cxs[i], cys[i] = oxi, oyi
                cxs[j], cys[j] = oxj, oyj
                no_imp += 1
                T *= alpha
                if T < Tmin: T = Tmin
                continue

        elif move_type == 5:
            # Bbox center move
            i = np.random.randint(0, n)
            ox, oy = cxs[i], cys[i]

            gx0, gy0, gx1, gy1 = get_global_bbox(cxs, cys, cangs, n)
            bcx, bcy = (gx0 + gx1) / 2, (gy0 + gy1) / 2
            dx, dy = bcx - cxs[i], bcy - cys[i]
            d = np.sqrt(dx*dx + dy*dy)
            if d > 1e-6:
                step = np.random.random() * move_scale * sc * 0.5
                cxs[i] += dx / d * step
                cys[i] += dy / d * step

            if check_overlap_single(i, cxs, cys, cangs, n):
                cxs[i], cys[i] = ox, oy
                no_imp += 1
                T *= alpha
                if T < Tmin: T = Tmin
                continue

        elif move_type == 6:
            # Corner tree focus - move trees that define bbox inward
            corner_indices, count = find_corner_trees(cxs, cys, cangs, n)
            if count > 0:
                idx = corner_indices[np.random.randint(0, count)]
                ox, oy, oa = cxs[idx], cys[idx], cangs[idx]

                gx0, gy0, gx1, gy1 = get_global_bbox(cxs, cys, cangs, n)
                bcx, bcy = (gx0 + gx1) / 2, (gy0 + gy1) / 2
                dx, dy = bcx - cxs[idx], bcy - cys[idx]
                d = np.sqrt(dx*dx + dy*dy)
                if d > 1e-6:
                    step = np.random.random() * move_scale * sc * 0.3
                    cxs[idx] += dx / d * step
                    cys[idx] += dy / d * step
                    cangs[idx] += (np.random.random() - 0.5) * rot_scale * sc * 0.5
                    cangs[idx] = cangs[idx] % 360

                if check_overlap_single(idx, cxs, cys, cangs, n):
                    cxs[idx], cys[idx], cangs[idx] = ox, oy, oa
                    no_imp += 1
                    T *= alpha
                    if T < Tmin: T = Tmin
                    continue
            else:
                no_imp += 1
                T *= alpha
                if T < Tmin: T = Tmin
                continue
        else:
            # Coordinated move - shift two adjacent trees together
            i = np.random.randint(0, n)
            j = (i + 1) % n

            oxi, oyi = cxs[i], cys[i]
            oxj, oyj = cxs[j], cys[j]

            dx = (np.random.random() - 0.5) * move_scale * sc * 0.5
            dy = (np.random.random() - 0.5) * move_scale * sc * 0.5

            cxs[i] += dx
            cys[i] += dy
            cxs[j] += dx
            cys[j] += dy

            if check_overlap_pair(i, j, cxs, cys, cangs, n):
                cxs[i], cys[i] = oxi, oyi
                cxs[j], cys[j] = oxj, oyj
                no_imp += 1
                T *= alpha
                if T < Tmin: T = Tmin
                continue

        ns = calc_side(cxs, cys, cangs, n)
        delta = ns - cs

        if delta < 0 or np.random.random() < np.exp(-delta / T):
            cs = ns
            if ns < bs:
                bs = ns
                bxs[:] = cxs
                bys[:] = cys
                bangs[:] = cangs
                no_imp = 0
            else:
                no_imp += 1
        else:
            cxs[:] = bxs
            cys[:] = bys
            cangs[:] = bangs
            cs = bs
            no_imp += 1

        # Reheat
        if no_imp > 600:
            T = min(T * 3.0, T0 * 0.7)
            no_imp = 0

        T *= alpha
        if T < Tmin:
            T = Tmin

    return bxs, bys, bangs, bs

In [ ]:
# --- CUDA GPU ACCELERATED KERNEL (OPTIONAL) ---
# This cell defines CUDA kernels for running Simulated Annealing on NVIDIA GPUs.
# It will only be used if a CUDA-compatible GPU is detected.

try:
    from numba import cuda
    from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
    
    # Constant memory for tree vertices
    # We can't easily use global constants in device functions without some setup, 
    # so we'll define them inside the device function or pass them.
    # For performance, hardcoding in the device function is often fastest for small arrays.
    
    @cuda.jit(device=True)
    def gpu_get_poly(cx, cy, deg, px_out, py_out):
        # Hardcoded vertices for the Christmas Tree
        # X: [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
        # Y: [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
        
        rad = deg * 3.141592653589793 / 180.0
        c = math.cos(rad)
        s = math.sin(rad)
        
        # Unrolled loop for 15 vertices
        # Vertex 0: (0, 0.8)
        px_out[0] = 0 * c - 0.8 * s + cx
        py_out[0] = 0 * s + 0.8 * c + cy
        # Vertex 1: (0.125, 0.5)
        px_out[1] = 0.125 * c - 0.5 * s + cx
        py_out[1] = 0.125 * s + 0.5 * c + cy
        # Vertex 2: (0.0625, 0.5)
        px_out[2] = 0.0625 * c - 0.5 * s + cx
        py_out[2] = 0.0625 * s + 0.5 * c + cy
        # Vertex 3: (0.2, 0.25)
        px_out[3] = 0.2 * c - 0.25 * s + cx
        py_out[3] = 0.2 * s + 0.25 * c + cy
        # Vertex 4: (0.1, 0.25)
        px_out[4] = 0.1 * c - 0.25 * s + cx
        py_out[4] = 0.1 * s + 0.25 * c + cy
        # Vertex 5: (0.35, 0.0)
        px_out[5] = 0.35 * c - 0.0 * s + cx
        py_out[5] = 0.35 * s + 0.0 * c + cy
        # Vertex 6: (0.075, 0.0)
        px_out[6] = 0.075 * c - 0.0 * s + cx
        py_out[6] = 0.075 * s + 0.0 * c + cy
        # Vertex 7: (0.075, -0.2)
        px_out[7] = 0.075 * c - (-0.2) * s + cx
        py_out[7] = 0.075 * s + (-0.2) * c + cy
        # Vertex 8: (-0.075, -0.2)
        px_out[8] = -0.075 * c - (-0.2) * s + cx
        py_out[8] = -0.075 * s + (-0.2) * c + cy
        # Vertex 9: (-0.075, 0.0)
        px_out[9] = -0.075 * c - 0.0 * s + cx
        py_out[9] = -0.075 * s + 0.0 * c + cy
        # Vertex 10: (-0.35, 0.0)
        px_out[10] = -0.35 * c - 0.0 * s + cx
        py_out[10] = -0.35 * s + 0.0 * c + cy
        # Vertex 11: (-0.1, 0.25)
        px_out[11] = -0.1 * c - 0.25 * s + cx
        py_out[11] = -0.1 * s + 0.25 * c + cy
        # Vertex 12: (-0.2, 0.25)
        px_out[12] = -0.2 * c - 0.25 * s + cx
        py_out[12] = -0.2 * s + 0.25 * c + cy
        # Vertex 13: (-0.0625, 0.5)
        px_out[13] = -0.0625 * c - 0.5 * s + cx
        py_out[13] = -0.0625 * s + 0.5 * c + cy
        # Vertex 14: (-0.125, 0.5)
        px_out[14] = -0.125 * c - 0.5 * s + cx
        py_out[14] = -0.125 * s + 0.5 * c + cy

    @cuda.jit(device=True)
    def gpu_get_bbox(px, py):
        minx = 1e9
        miny = 1e9
        maxx = -1e9
        maxy = -1e9
        for i in range(15):
            if px[i] < minx: minx = px[i]
            if px[i] > maxx: maxx = px[i]
            if py[i] < miny: miny = py[i]
            if py[i] > maxy: maxy = py[i]
        return minx, miny, maxx, maxy

    @cuda.jit(device=True)
    def gpu_pip(px_pt, py_pt, poly_x, poly_y):
        inside = False
        j = 14
        for i in range(15):
            if ((poly_y[i] > py_pt) != (poly_y[j] > py_pt) and
                px_pt < (poly_x[j] - poly_x[i]) * (py_pt - poly_y[i]) / (poly_y[j] - poly_y[i]) + poly_x[i]):
                inside = not inside
            j = i
        return inside

    @cuda.jit(device=True)
    def gpu_seg_intersect(ax, ay, bx, by, cx, cy, dx, dy):
        # CCW function inline
        ccw1 = (cy - ay) * (bx - ax) > (by - ay) * (cx - ax)
        ccw2 = (dy - ay) * (bx - ax) > (by - ay) * (dx - ax)
        ccw3 = (cy - cx) * (dx - cx) > (dy - cx) * (ax - cx)
        ccw4 = (cy - cx) * (dx - cx) > (dy - cx) * (bx - cx)
        return (ccw1 != ccw2) and (ccw3 != ccw4)

    @cuda.jit(device=True)
    def gpu_overlap(px1, py1, minx1, miny1, maxx1, maxy1, px2, py2, minx2, miny2, maxx2, maxy2):
        # BBox check
        if maxx1 < minx2 or maxx2 < minx1 or maxy1 < miny2 or maxy2 < miny1:
            return False
            
        # Point in Polygon
        for i in range(15):
            if gpu_pip(px1[i], py1[i], px2, py2): return True
            if gpu_pip(px2[i], py2[i], px1, py1): return True
            
        # Edge Intersection
        for i in range(15):
            ni = (i + 1) % 15
            for j in range(15):
                nj = (j + 1) % 15
                if gpu_seg_intersect(px1[i], py1[i], px1[ni], py1[ni],
                                   px2[j], py2[j], px2[nj], py2[nj]):
                    return True
        return False

    @cuda.jit
    def sa_gpu_kernel(states_x, states_y, states_ang, rng_states, iterations, T0, Tmin, move_scale, rot_scale, scores_out):
        # Each thread handles one configuration (one row in states)
        tid = cuda.grid(1)
        if tid >= states_x.shape[0]:
            return
            
        n = states_x.shape[1]
        
        # Local arrays for geometry (stack allocated)
        px1 = cuda.local.array(15, dtype=np.float64)
        py1 = cuda.local.array(15, dtype=np.float64)
        px2 = cuda.local.array(15, dtype=np.float64)
        py2 = cuda.local.array(15, dtype=np.float64)
        
        # Load state into registers/local memory if possible, but for N=200 it's too big for registers.
        # We will operate directly on global memory or cache.
        # To avoid global memory latency, we could cache, but N varies.
        # For simplicity, we operate on global memory `states_x[tid, :]`.
        
        # Calculate initial score
        gx0, gy0, gx1, gy1 = 1e9, 1e9, -1e9, -1e9
        for i in range(n):
            gpu_get_poly(states_x[tid, i], states_y[tid, i], states_ang[tid, i], px1, py1)
            minx, miny, maxx, maxy = gpu_get_bbox(px1, py1)
            if minx < gx0: gx0 = minx
            if miny < gy0: gy0 = miny
            if maxx > gx1: gx1 = maxx
            if maxy > gy1: gy1 = maxy
            
        current_side = max(gx1 - gx0, gy1 - gy0)
        best_side = current_side
        
        # Save best state (we can just overwrite if we improve, or keep a separate best buffer)
        # For this kernel, we'll just update the state in place to the current state.
        # If we want to track "best ever", we need more memory. 
        # Standard SA tracks "current" and "best". 
        # Let's assume states_x is "current" and we just return the final "current".
        
        T = T0
        alpha = (Tmin / T0) ** (1.0 / iterations)
        
        for it in range(iterations):
            # Pick a move
            move_type = int(xoroshiro128p_uniform_float32(rng_states, tid) * 4) # 0-3
            
            # Select tree
            idx = int(xoroshiro128p_uniform_float32(rng_states, tid) * n)
            if idx >= n: idx = n - 1
            
            old_x = states_x[tid, idx]
            old_y = states_y[tid, idx]
            old_ang = states_ang[tid, idx]
            
            # Propose move
            sc = T / T0
            if move_type == 0: # Translate
                states_x[tid, idx] += (xoroshiro128p_uniform_float32(rng_states, tid) - 0.5) * 2 * move_scale * sc
                states_y[tid, idx] += (xoroshiro128p_uniform_float32(rng_states, tid) - 0.5) * 2 * move_scale * sc
            elif move_type == 1: # Rotate
                states_ang[tid, idx] += (xoroshiro128p_uniform_float32(rng_states, tid) - 0.5) * 2 * rot_scale * sc
            else: # Mixed
                states_x[tid, idx] += (xoroshiro128p_uniform_float32(rng_states, tid) - 0.5) * move_scale * sc
                states_y[tid, idx] += (xoroshiro128p_uniform_float32(rng_states, tid) - 0.5) * move_scale * sc
                states_ang[tid, idx] += (xoroshiro128p_uniform_float32(rng_states, tid) - 0.5) * rot_scale * sc
                
            # Check overlap
            overlap_found = False
            gpu_get_poly(states_x[tid, idx], states_y[tid, idx], states_ang[tid, idx], px1, py1)
            minx1, miny1, maxx1, maxy1 = gpu_get_bbox(px1, py1)
            
            for j in range(n):
                if idx == j: continue
                gpu_get_poly(states_x[tid, j], states_y[tid, j], states_ang[tid, j], px2, py2)
                minx2, miny2, maxx2, maxy2 = gpu_get_bbox(px2, py2)
                
                if gpu_overlap(px1, py1, minx1, miny1, maxx1, maxy1, px2, py2, minx2, miny2, maxx2, maxy2):
                    overlap_found = True
                    break
            
            if overlap_found:
                # Revert
                states_x[tid, idx] = old_x
                states_y[tid, idx] = old_y
                states_ang[tid, idx] = old_ang
            else:
                # Calculate new score
                # This is expensive (O(N)), but necessary for exact SA. 
                # Optimization: Update bounds incrementally? Hard with rotation.
                # We do full recalc.
                gx0, gy0, gx1, gy1 = 1e9, 1e9, -1e9, -1e9
                for i in range(n):
                    gpu_get_poly(states_x[tid, i], states_y[tid, i], states_ang[tid, i], px1, py1)
                    minx, miny, maxx, maxy = gpu_get_bbox(px1, py1)
                    if minx < gx0: gx0 = minx
                    if miny < gy0: gy0 = miny
                    if maxx > gx1: gx1 = maxx
                    if maxy > gy1: gy1 = maxy
                
                new_side = max(gx1 - gx0, gy1 - gy0)
                delta = new_side - current_side
                
                if delta < 0 or xoroshiro128p_uniform_float32(rng_states, tid) < math.exp(-delta / T):
                    current_side = new_side
                    if current_side < best_side:
                        best_side = current_side
                else:
                    # Revert
                    states_x[tid, idx] = old_x
                    states_y[tid, idx] = old_y
                    states_ang[tid, idx] = old_ang
            
            T *= alpha
            if T < Tmin: T = Tmin
            
        scores_out[tid] = best_side

    print("CUDA Kernels compiled successfully.")

except ImportError:
    print("CUDA not available. GPU kernels skipped.")
    
except Exception as e:
    print(f"Error compiling CUDA kernels: {e}")


In [ ]:
# Load and evaluate the existing test.csv to verify the score
import pandas as pd
try:
    test_df = pd.read_csv('test.csv')
    print("Loaded test.csv")
    
    # Parse the 's' prefix
    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    test_df['x'] = test_df['x'].apply(parse_s)
    test_df['y'] = test_df['y'].apply(parse_s)
    test_df['deg'] = test_df['deg'].apply(parse_s)
    
    # Reconstruct trees and calculate score
    total_test_score = 0
    for n in range(1, 201):
        # Get rows for this N
        # The ID format is N_i, e.g., 001_0
        # We need to filter by the prefix
        prefix = f"{n:03d}_"
        rows = test_df[test_df['id'].str.startswith(prefix)]
        
        if len(rows) != n:
            print(f"Warning: N={n} has {len(rows)} rows, expected {n}")
            continue
            
        trees = []
        for _, row in rows.iterrows():
            trees.append(ChristmasTree(row['x'], row['y'], row['deg']))
            
        side = get_bounds(trees)
        score_n = (side ** 2) / n
        total_test_score += score_n
        
    print(f"Score of test.csv: {total_test_score:.4f}")
    
except Exception as e:
    print(f"Could not evaluate test.csv: {e}")


## 6. Model Training (Optional)

For this geometric packing problem, standard supervised learning is less applicable than search algorithms. However, one could train a model to predict the optimal *order* of placement or the optimal *angle* given the current boundary shape. We skip this for the baseline.

In [ ]:
# Placeholder for ML model training
# model = LGBMRegressor(...)
# model.fit(X_train, y_train)

## 7. Optimization Strategy

We define a modular optimization function. Currently, it's a placeholder for a more advanced local search (e.g., trying to wiggle trees after placement).

In [ ]:
def center_packing(trees):
    """Centers the packing at (0,0) based on the bounding box center."""
    if not trees: return trees
    minx, miny, maxx, maxy = unary_union([t.polygon for t in trees]).bounds
    cx = (minx + maxx) / 2
    cy = (miny + maxy) / 2
    
    # If already centered (small epsilon), skip
    if abs(cx) < 1e-9 and abs(cy) < 1e-9:
        return trees
        
    for t in trees:
        t.update_position(t.center_x - cx, t.center_y - cy, t.angle)
    return trees

def optimize_packing(trees, params, target_side=None):
    """
    Simulated Annealing with Numba Acceleration.
    """
    if not trees: return trees
    
    n = len(trees)
    
    # Extract data for Numba
    xs = np.array([t.center_x for t in trees], dtype=np.float64)
    ys = np.array([t.center_y for t in trees], dtype=np.float64)
    angs = np.array([t.angle for t in trees], dtype=np.float64)
    
    # Parameters
    iterations = params.get('iterations', 10000)
    T0 = params.get('initial_temp', 1.0)
    Tmin = params.get('final_temp', 1e-6)
    move_scale = params.get('step_size', 0.5)
    rot_scale = params.get('angle_step', 10.0)
    
    # Generate a random seed for this run
    seed = np.random.randint(0, 1000000)
    
    # Run Numba SA
    # First run might be slow due to JIT compilation
    best_xs, best_ys, best_angs, best_side = sa_numba(
        xs, ys, angs, n, iterations, T0, Tmin, move_scale, rot_scale, seed
    )
    
    # Update trees
    for i in range(n):
        trees[i].update_position(best_xs[i], best_ys[i], best_angs[i])
        
    return center_packing(trees)


## 8. Submission Generation

This section runs the full pipeline and generates the submission file.

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import os
import datetime
import time

# Load baseline from test.csv
known_solutions = {}
try:
    print("Loading baseline from test.csv...")
    baseline_df = pd.read_csv('test.csv')
    
    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    baseline_df['x'] = baseline_df['x'].apply(parse_s)
    baseline_df['y'] = baseline_df['y'].apply(parse_s)
    baseline_df['deg'] = baseline_df['deg'].apply(parse_s)
    
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = baseline_df[baseline_df['id'].str.startswith(prefix)]
        if len(rows) == n:
            trees = []
            for _, row in rows.iterrows():
                trees.append(ChristmasTree(row['x'], row['y'], row['deg']))
            known_solutions[n] = trees
            
    print(f"Loaded {len(known_solutions)} configurations from test.csv")
    
except Exception as e:
    print(f"Could not load baseline: {e}")

# --- GPU HELPER FUNCTIONS ---
def run_gpu_batch_optimization(candidates, n, iterations=10000):
    """
    Runs SA on a batch of candidates using the GPU.
    candidates: List of list of ChristmasTree objects.
    Returns: List of (score, trees) tuples.
    """
    try:
        from numba import cuda
        from numba.cuda.random import create_xoroshiro128p_states
        
        batch_size = len(candidates)
        if batch_size == 0: return []
        
        # Prepare data arrays
        # Shape: (batch_size, n)
        h_xs = np.zeros((batch_size, n), dtype=np.float64)
        h_ys = np.zeros((batch_size, n), dtype=np.float64)
        h_angs = np.zeros((batch_size, n), dtype=np.float64)
        
        for i, trees in enumerate(candidates):
            for j, tree in enumerate(trees):
                h_xs[i, j] = tree.center_x
                h_ys[i, j] = tree.center_y
                h_angs[i, j] = tree.angle
                
        # Copy to device
        d_xs = cuda.to_device(h_xs)
        d_ys = cuda.to_device(h_ys)
        d_angs = cuda.to_device(h_angs)
        d_scores = cuda.device_array(batch_size, dtype=np.float64)
        
        # RNG States
        rng_states = create_xoroshiro128p_states(batch_size, seed=random.randint(0, 1000000))
        
        # Launch Kernel
        threads_per_block = 64
        blocks = (batch_size + threads_per_block - 1) // threads_per_block
        
        # Params
        T0 = 1.0
        Tmin = 1e-5
        move_scale = 0.5
        rot_scale = 10.0
        
        sa_gpu_kernel[blocks, threads_per_block](
            d_xs, d_ys, d_angs, rng_states, iterations, T0, Tmin, move_scale, rot_scale, d_scores
        )
        cuda.synchronize()
        
        # Copy back
        h_xs_out = d_xs.copy_to_host()
        h_ys_out = d_ys.copy_to_host()
        h_angs_out = d_angs.copy_to_host()
        h_scores_out = d_scores.copy_to_host()
        
        results = []
        for i in range(batch_size):
            new_trees = []
            for j in range(n):
                new_trees.append(ChristmasTree(h_xs_out[i, j], h_ys_out[i, j], h_angs_out[i, j]))
            results.append((h_scores_out[i], new_trees))
            
        return results
        
    except Exception as e:
        print(f"GPU Execution Failed: {e}")
        return []

# --- CPU HELPER FUNCTIONS ---
def process_beam_candidate_cpu(base_trees, n, packer_params, target_side=None, remove_idx=None, parent_id=None):
    # Re-seed random
    seed_val = (int(time.time() * 1000000) + os.getpid() + (remove_idx if remove_idx is not None else 0)) % (2**32)
    random.seed(seed_val)
    np.random.seed(seed_val)
    
    current_trees = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in base_trees]
    
    # Remove trees if needed
    while len(current_trees) > n:
        if remove_idx is not None and remove_idx < len(current_trees):
            current_trees.pop(remove_idx)
            remove_idx = None
        else:
            # Fallback removal
            idx = random.randint(0, len(current_trees) - 1)
            current_trees.pop(idx)
            
    # Optimize
    iterations = max(200000, min(1000000, int(n * 5000)))
    candidate_trees = optimize_packing(current_trees, {
        'iterations': iterations,
        'step_size': 1.0,
        'angle_step': 45.0,
        'initial_temp': 2.0,
        'final_temp': 1e-5,
        'compression': 0.1
    }, target_side=target_side)
        
    side_candidate = get_bounds(candidate_trees)
    score_candidate = (side_candidate ** 2) / n
    
    return (score_candidate, candidate_trees, parent_id)

# BEAM SEARCH SETUP
BASE_BEAM_WIDTH = CONFIG['dynamic_beam_width']['base_width']
BRANCH_FACTOR = 8
n_jobs = multiprocessing.cpu_count()

# Check for GPU
USE_GPU = False
try:
    from numba import cuda
    if cuda.is_available():
        USE_GPU = True
        print(f"CUDA GPU Detected: {cuda.get_current_device().name}")
        print("Using GPU for optimization batching.")
    else:
        print("No CUDA GPU detected. Using CPU multiprocessing.")
except:
    print("No CUDA GPU detected. Using CPU multiprocessing.")

packer_params = {'n_trials': 200, 'step_size': 0.2, 'fine_step': 0.01}
submission_rows = []
improvements = 0
all_solutions = {}

# Initialize candidates
if 200 in known_solutions:
    current_candidates = [(known_solutions[200], "init")]
else:
    current_candidates = []

# Metrics Tracking
history_n = []
history_improvement = []
history_total_score = []
current_total_score = 0

# Setup Metrics File
os.makedirs('Data', exist_ok=True)
timestamp_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
metrics_file = f'Data/run_metrics_advanced_{timestamp_str}.csv'
with open(metrics_file, 'w') as f:
    f.write('n,score,baseline,improvement,source\n')

# Setup Realtime Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
plt.close(fig)
plot_display_id = "metrics_plot_advanced"
display(fig, display_id=plot_display_id)

# REVERSE LOOP
for n in tqdm(range(200, 0, -1), desc="Processing Reverse"):
    print(f"\n--- Processing N={n} ---")

    # Dynamic Beam Width
    current_beam_width = BASE_BEAM_WIDTH
    if CONFIG['dynamic_beam_width']['enabled']:
        if n in [100, 50, 25, 10] or n <= 5:
            current_beam_width = int(BASE_BEAM_WIDTH * CONFIG['dynamic_beam_width']['critical_n_multiplier'])

    # 1. Generate Candidates (Removal Phase)
    # We do this on CPU because it's fast and requires complex logic (shapely)
    candidates_to_optimize = [] # List of (trees, parent_id)
    
    for p_idx, (base_trees, _) in enumerate(current_candidates):
        # Identify boundary trees
        minx, miny, maxx, maxy = unary_union([t.polygon for t in base_trees]).bounds
        boundary_indices = []
        for i, t in enumerate(base_trees):
            tb = t.polygon.bounds
            if (abs(tb[0] - minx) < 1e-2 or abs(tb[1] - miny) < 1e-2 or 
                abs(tb[2] - maxx) < 1e-2 or abs(tb[3] - maxy) < 1e-2):
                boundary_indices.append(i)
        
        indices_to_try = boundary_indices[:BRANCH_FACTOR]
        while len(indices_to_try) < BRANCH_FACTOR:
            indices_to_try.append(None)
            
        for idx in indices_to_try:
            # Create removed version
            temp_trees = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in base_trees]
            if idx is not None and idx < len(temp_trees):
                temp_trees.pop(idx)
            else:
                # Random removal
                temp_trees.pop(random.randint(0, len(temp_trees)-1))
            
            # Ensure size is n (handle case where base was > n+1)
            while len(temp_trees) > n:
                temp_trees.pop()
                
            candidates_to_optimize.append((temp_trees, p_idx))

    # 2. Optimize Candidates (GPU or CPU)
    results = []
    
    if USE_GPU:
        # GPU Batch Optimization
        # We can run multiple restarts for each candidate to fill the GPU
        gpu_batch = []
        gpu_metadata = [] # (parent_id)
        
        RESTARTS_PER_CANDIDATE = 16 # Increase parallelism
        
        for trees, pid in candidates_to_optimize:
            for _ in range(RESTARTS_PER_CANDIDATE):
                # Deep copy for restart
                gpu_batch.append([ChristmasTree(t.center_x, t.center_y, t.angle) for t in trees])
                gpu_metadata.append(pid)
                
        print(f"  [N={n}] Launching GPU Kernel with {len(gpu_batch)} threads...")
        gpu_results = run_gpu_batch_optimization(gpu_batch, n, iterations=200000)
        
        for i, (score, trees) in enumerate(gpu_results):
            results.append((score, trees, gpu_metadata[i]))
            
    else:
        # CPU Parallel Optimization
        print(f"  [N={n}] Launching {len(candidates_to_optimize)} CPU tasks...")
        tasks = []
        for trees, pid in candidates_to_optimize:
            tasks.append((trees, n, packer_params, None, None, pid))
            
        # Note: process_beam_candidate_cpu expects base_trees > n, but we already reduced them.
        # We need a slight adapter or just use optimize_packing directly in parallel.
        # Let's use a simple wrapper.
        def run_cpu_opt(trees, pid):
            opt_trees = optimize_packing(trees, {
                'iterations': 200000,
                'step_size': 1.0,
                'angle_step': 45.0,
                'initial_temp': 2.0,
                'final_temp': 1e-5
            })
            s = get_bounds(opt_trees)
            return ((s**2)/n, opt_trees, pid)
            
        results = Parallel(n_jobs=n_jobs)(
            delayed(run_cpu_opt)(t[0], t[1]) for t in candidates_to_optimize
        )

    # 3. Sort and Select
    results.sort(key=lambda x: x[0])
    if not results:
        print(f"  [N={n}] No results!")
        continue
        
    print(f"  [N={n}] Best candidate score: {results[0][0]:.6f}")
    
    # Diversity Selection
    unique_candidates = []
    seen_scores = set()
    parent_counts = {}
    max_children = CONFIG['beam_diversity']['max_children_per_parent']
    
    for res in results:
        score = round(res[0], 6)
        trees = res[1]
        pid = res[2]
        
        if score in seen_scores: continue
        if parent_counts.get(pid, 0) >= max_children: continue
        
        unique_candidates.append((trees, pid))
        seen_scores.add(score)
        parent_counts[pid] = parent_counts.get(pid, 0) + 1
        
        if len(unique_candidates) >= current_beam_width:
            break
            
    current_candidates = unique_candidates
    best_trees_n = results[0][1]
    best_score_n = results[0][0]
    source = "Reverse"
    
    # 4. Baseline Comparison
    baseline_val = 0
    if n in known_solutions:
        known_trees = center_packing(known_solutions[n])
        side_known = get_bounds(known_trees)
        score_known = (side_known ** 2) / n
        baseline_val = score_known
        
        if score_known < best_score_n - 1e-7:
            print(f"  [N={n}] Baseline is better ({score_known:.6f} < {best_score_n:.6f}).")
            best_trees_n = known_trees
            best_score_n = score_known
            source = "Baseline"
            
            # Try to optimize baseline on GPU/CPU
            if USE_GPU:
                # Run massive GPU optimization on baseline
                print(f"  [N={n}] Optimizing baseline on GPU...")
                base_batch = []
                for _ in range(128): # 128 restarts
                    base_batch.append([ChristmasTree(t.center_x, t.center_y, t.angle) for t in known_trees])
                
                base_results = run_gpu_batch_optimization(base_batch, n, iterations=500000)
                base_results.sort(key=lambda x: x[0])
                if base_results[0][0] < score_known - 1e-7:
                    best_trees_n = base_results[0][1]
                    best_score_n = base_results[0][0]
                    source = "Baseline+GPU"
                    improvements += 1
                    print(f"  [N={n}] GPU Improved Baseline! -> {best_score_n:.6f}")
            else:
                # CPU Optimization
                opt_known = optimize_packing(known_trees, {'iterations': 500000, 'step_size': 0.8})
                s_opt = (get_bounds(opt_known)**2)/n
                if s_opt < score_known - 1e-7:
                    best_trees_n = opt_known
                    best_score_n = s_opt
                    source = "Baseline+CPU"
                    improvements += 1
                    print(f"  [N={n}] CPU Improved Baseline! -> {best_score_n:.6f}")
                    
            # Inject back into beam
            current_candidates.append((best_trees_n, "baseline"))
            
        elif best_score_n < score_known - 1e-7:
            print(f"  [N={n}] Improvement found! {score_known:.6f} -> {best_score_n:.6f}")
            improvements += 1
            
    # 5. Save & Plot
    all_solutions[n] = best_trees_n
    current_total_score += best_score_n
    imp = max(0, baseline_val - best_score_n) if baseline_val > 0 else 0
    
    with open(metrics_file, 'a') as f:
        f.write(f"{n},{best_score_n:.10f},{baseline_val:.10f},{imp:.10f},{source}\n")
        
    if n % 5 == 0 or n == 1:
        ax1.clear(); ax2.clear()
        history_n.append(n); history_improvement.append(imp); history_total_score.append(current_total_score)
        ax1.plot(history_n, history_improvement, 'g-'); ax1.set_title(f'Improvement (Total: {improvements})'); ax1.invert_xaxis()
        ax2.plot(history_n, history_total_score, 'b-'); ax2.set_title(f'Score: {current_total_score:.4f}'); ax2.invert_xaxis()
        display(fig, display_id=plot_display_id, update=True)
        
    # Prepare submission rows
    for i, tree in enumerate(best_trees_n):
        submission_rows.append([f"{n:03d}_{i}", f"s{tree.center_x:.10f}", f"s{tree.center_y:.10f}", f"s{tree.angle:.10f}"])

print(f"Processing complete. Improvements: {improvements}")
df_sub = pd.DataFrame(submission_rows, columns=['id', 'x', 'y', 'deg'])
df_sub.sort_values('id', inplace=True)
df_sub.to_csv('submission.csv', index=False)
print("Submission generated.")

## 10. Evaluation Helper

Calculate the local score to estimate leaderboard performance.


In [ ]:
## 9. Polishing & Refinement
# Run this cell to further optimize the existing submission using the new Numba engine.
# This is much faster than the full search and can squeeze out extra points.

print("Starting Polishing Phase...")
try:
    # Load current best submission
    if os.path.exists('submission.csv'):
        polish_df = pd.read_csv('submission.csv')
        print("Loaded submission.csv for polishing.")
    else:
        polish_df = pd.read_csv('test.csv')
        print("Loaded test.csv for polishing.")

    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    polish_df['x'] = polish_df['x'].apply(parse_s)
    polish_df['y'] = polish_df['y'].apply(parse_s)
    polish_df['deg'] = polish_df['deg'].apply(parse_s)
    
    polish_solutions = {}
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = polish_df[polish_df['id'].str.startswith(prefix)]
        if len(rows) == n:
            trees = []
            for _, row in rows.iterrows():
                trees.append(ChristmasTree(row['x'], row['y'], row['deg']))
            polish_solutions[n] = trees

    total_score_before = 0
    total_score_after = 0
    
    # Polish each N
    for n in tqdm(range(1, 201), desc="Polishing"):
        if n not in polish_solutions: continue
        
        trees = polish_solutions[n]
        side_before = get_bounds(trees)
        score_before = (side_before ** 2) / n
        total_score_before += score_before
        
        # Run Numba Optimization
        # High iterations because it's fast!
        iters = 200000 if n < 50 else 500000
        
        optimized_trees = optimize_packing(trees, {
            'iterations': iters,
            'step_size': 0.5,
            'angle_step': 15.0,
            'initial_temp': 1.0,
            'final_temp': 1e-6
        })
        
        side_after = get_bounds(optimized_trees)
        score_after = (side_after ** 2) / n
        
        if score_after < score_before - 1e-9:
            polish_solutions[n] = optimized_trees
            total_score_after += score_after
            # print(f"  N={n} Improved: {score_before:.6f} -> {score_after:.6f}")
        else:
            total_score_after += score_before # Keep original
            
    print(f"Polishing Complete.")
    print(f"Score Before: {total_score_before:.6f}")
    print(f"Score After:  {total_score_after:.6f}")
    print(f"Improvement:  {total_score_before - total_score_after:.6f}")
    
    # Save if improved
    if total_score_after < total_score_before:
        new_rows = []
        for n in range(1, 201):
            if n in polish_solutions:
                for i, tree in enumerate(polish_solutions[n]):
                    new_rows.append([
                        f"{n:03d}_{i}", 
                        f"s{tree.center_x:.10f}", 
                        f"s{tree.center_y:.10f}", 
                        f"s{tree.angle:.10f}"
                    ])
        
        df_polished = pd.DataFrame(new_rows, columns=['id', 'x', 'y', 'deg'])
        df_polished.sort_values('id', inplace=True)
        df_polished.to_csv('submission_polished.csv', index=False)
        df_polished.to_csv('submission.csv', index=False) # Overwrite main
        print("Saved polished submission to submission.csv")
        
except Exception as e:
    print(f"Polishing failed: {e}")


In [ ]:
import os
import datetime

# Calculate final score
final_score = 0
for n, trees in all_solutions.items():
    side = get_bounds(trees)
    final_score += (side ** 2) / n

print(f"Final Score: {final_score:.10f}")

# Compare with original
if 'total_test_score' in globals():
    print(f"Original Score: {total_test_score:.10f}")
    if final_score < total_test_score:
        diff = total_test_score - final_score
        print(f"SUCCESS: Score improved by {diff:.10f}!")
        
        # 1. Save copy with detailed name
        os.makedirs('Data', exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        detailed_name = f"Data/submission_score{final_score:.2f}_improved{diff:.2f}_{timestamp}.csv"
        df_sub.to_csv(detailed_name, index=False)
        print(f"Saved backup: {detailed_name}")
        
        # 2. Overwrite test.csv
        df_sub.to_csv('test.csv', index=False)
        print("Overwrote test.csv")
        
        # 3. Submit to Kaggle
        message = f"Improved score {final_score:.6f} (was {total_test_score:.6f})"
        print("Submitting to Kaggle...")
        !kaggle competitions submit -c santa-2025 -f submission.csv -m "{message}"

        # 4. Git Commit and Push
        print("Committing and pushing to Git...")
        !git add .
        !git commit -m "{message}"
        !git push
        
    else:
        print(f"No improvement (Current: {final_score:.10f} >= Original: {total_test_score:.10f}).")
else:
    print("Original score not found. Run the first cell to load test.csv baseline.")